# Transform

In this notebook we're going to Transform GPX files stored in Digital Ocean spaces.  If you notice, this project is missing an extract step. That's because I have a process setup to upload Equilab data using a Shotrcut on my phone, which is essentially my extract process. Equilab doens't have an API, so I can't extract the data in the traditional sense.

Before we get to the fun stuff first we must import:

## Import

In [63]:
import bucketstore
from geopy import distance as geodistance
import geopy
import yaml
import json


## Setup

With importing out of the way we'll get a few basics out of the way. The below cell:
 1) Loads a file containing connection info to my DO Space
 2) Uses the connection info to connect to the space
 3) Loads the bucket that all of the data is stored in.

In [64]:
with open("secrets.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

bucketstore.login(
    access_key_id=cfg['spaces']['access'],
    secret_access_key=cfg['spaces']['secret'],
    region='nyc3',
    endpoint_url=cfg['spaces']['url']
)
bucket = bucketstore.get('wrathalake')

ridesKey = 'intermediate/rides.json'

## Functions

To keep the code clean we're going to encapsulate a few things in functions.

In [65]:
def processFile(key):
    gpx = gpxpy.parse(bucket[key])
    
    # Choosing to store coordinates in a dict to remove ambiguity. Apparently we can't all agree to use long, lat or lat, long
    data = [{
        'time': point.time,
        'coords': {
            'long': point.longitude,
            'lat': point.latitude
        },
        'elevation': point.elevation * 3.28084
    } for point in gpx.tracks[0].segments[0].points]
    
    # Calculating some ride metrics
    totalDistance = 0
    totalTime = (data[-1]['time'] - data[0]['time']).total_seconds()
    totalClimb = 0
    
    prv = nxt = None
    l = len(data)
    
    for index, obj in enumerate(data):
        if index > 0:
            prv = data[index - 1]

        if index < (l - 1):
            nxt = data[index + 1]

        if prv is not None:
            timeDelta = (obj['time'] - prv['time']).total_seconds()
            
            # geopy uses coordinates in the (lat, long) format, so we'll create that below.
            distance = geodistance.geodesic((obj['coords']['lat'], obj['coords']['long']), (prv['coords']['lat'], prv['coords']['long'])).ft
            totalDistance += distance
            
            speed = (distance / timeDelta) * 0.681818182
            
            if obj['elevation'] > prv['elevation']:
                climb = obj['elevation'] - prv['elevation']
                totalClimb += climb
            else:
                climb = None

            obj['timeDelta'] = timeDelta
            obj['distance'] = distance
            obj['speed'] = speed
            obj['climb'] = climb
            obj['drop'] = (prv['elevation'] - obj['elevation']) if obj['elevation'] < prv['elevation'] else None
        
    rideData = {
        'rawFile': key,
        'interFile': key.replace('raw', 'intermediate').replace('gpx', 'json'),
        'rideDate': data[0]['time'].strftime("%Y%m%d"),
        'totalTime': totalTime,
        'totalDistance': totalDistance,
        'totalClimb': totalClimb,
        'averageSpeed': (totalDistance / totalTime) * 0.681818182,
        'inElastic': False
    }
    
    # Once we're done processing we need to set the time to be something that can be stored in JSON
    for index, obj in enumerate(data):
        timestamp = obj['time'].isoformat()
        obj['time'] = timestamp
            
    return data, rideData


## History

When ever this process runs it uses a Rides file that documents each ride. If a ride is listed in this file it means that it has already been processed and thus doesn't need processed again.

So first things first, let's load the rides file.

In [66]:
try:
    rides = json.loads(bucket['intermediate/rides.json'])
except:
    bucket[ridesKey] = json.dumps({})
    rides = {}

rideKeys = rides.keys()

print('There are {} rides that have been processed.'.format(len(rides.keys())))

There are 0 rides that have been processed.


## Raw Ride Files

Now that we have a list of files that have been processed we can check all of the files against it and process just the ones that need it.

In [67]:
objects = bucket.list(prefix='raw/sources/equilab/')


In [68]:
for obj in objects:
    if obj.endswith('.gpx') and obj not in rideKeys:
        print('Processing {}'.format(obj))
        fileData, rideData = processFile(obj)
        
        newKey = obj.replace('raw', 'intermediate').replace('gpx', 'json')
        bucket[newKey] = json.dumps(fileData)
        
        rides[obj] = rideData

bucket[ridesKey] = json.dumps(rides)

Processing raw/sources/equilab/training-2023-09-09.gpx
Processing raw/sources/equilab/training-2023-09-10.gpx
Processing raw/sources/equilab/training-2023-09-23.gpx
Processing raw/sources/equilab/training-2023-10-14.gpx
Processing raw/sources/equilab/training-2023-10-22.gpx


## Helper Functions

The below lines of code are meant as helper functions and in general they should be left commented out. But it's not always a bad thing to leave them uncommented and save them as history for the scheduled process that runs the notebook.


In [23]:
# View the rides JSON file
# bucket[ridesKey]


In [62]:
# Delete the rides file, which essentially will restart processing
# del bucket[ridesKey]

In [24]:
# List all of the ride data in the intermediate directory
# bucket.list(prefix='intermediate/sources/equilab/')


In [25]:
# Delete all of the processes rides
# for key in bucket.list(prefix='intermediate/sources/equilab/'):
#     del bucket[key]


In [69]:
# View a few lines from the last fileData processed
fileData[:3]


[{'time': '2023-10-22T19:14:40.014000+00:00',
  'coords': {'long': -86.0489782, 'lat': 37.9405569},
  'elevation': 659.120756},
 {'time': '2023-10-22T19:14:41+00:00',
  'coords': {'long': -86.0489829, 'lat': 37.9405528},
  'elevation': 659.776924,
  'timeDelta': 0.986,
  'distance': 2.0165503741423745,
  'speed': 1.3944429107598109,
  'climb': 0.6561679999999797,
  'drop': None},
 {'time': '2023-10-22T19:14:42+00:00',
  'coords': {'long': -86.0489954, 'lat': 37.9405522},
  'elevation': 664.042016,
  'timeDelta': 1.0,
  'distance': 3.6115741930671343,
  'speed': 2.4624369504751504,
  'climb': 4.2650919999999815,
  'drop': None}]

In [70]:
# View the rides data JSON
rides


{'raw/sources/equilab/training-2023-09-09.gpx': {'rawFile': 'raw/sources/equilab/training-2023-09-09.gpx',
  'interFile': 'intermediate/sources/equilab/training-2023-09-09.json',
  'rideDate': '20230909',
  'totalTime': 6526.014,
  'totalDistance': 26598.70310297059,
  'totalClimb': 8746.719439999997,
  'averageSpeed': 2.778951959530759,
  'inElastic': False},
 'raw/sources/equilab/training-2023-09-10.gpx': {'rawFile': 'raw/sources/equilab/training-2023-09-10.gpx',
  'interFile': 'intermediate/sources/equilab/training-2023-09-10.json',
  'rideDate': '20230910',
  'totalTime': 7021.024,
  'totalDistance': 26223.356190037925,
  'totalClimb': 5603.346635999986,
  'averageSpeed': 2.5465745514372413,
  'inElastic': False},
 'raw/sources/equilab/training-2023-09-23.gpx': {'rawFile': 'raw/sources/equilab/training-2023-09-23.gpx',
  'interFile': 'intermediate/sources/equilab/training-2023-09-23.json',
  'rideDate': '20230923',
  'totalTime': 11583.061,
  'totalDistance': 39063.74337117145,
  '